## Model
**This notebook will do the following:**
1. Expand the CSV file.
2. Convert passivating molecules into SMILES representations and extract their features.
3. Retrieve the composition of the perovskite.

(work in progress)

4. Train a model using the dataset.
5. Predict new PCE values for different pairings of passivating molecules and perovskites. 

In [100]:
import pandas as pd
import ast
import json
import numpy as np
import pubchempy as pcp
from rdkit import Chem
from rdkit.Chem import Descriptors, rdMolDescriptors

import requests
import re

import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors, rdMolDescriptors

### Load & expand json into DataFrame rows + Selecting

In [3]:
papers_df = pd.read_csv('150_papers_json_update.csv')
papers_df.head()

,first_num,id,text,memory,output,second_num
0,0,0_54,\t\t\t of 5 Downloaded from https://www.scienc...,"{""perovskite_composition"": ""Cs0.05FA0.85MA0.1P...","{""perovskite_composition"": ""Cs0.05FA0.85MA0.1P...",54
1,1,1_22,\t\t\t NAture PhotoNiCS | VOL 13 | JULY 2019 |...,"{""perovskite_composition"": null, ""electron_tra...","{""perovskite_composition"": null, ""electron_tra...",22
2,2,2_75,\t\t\t Nature eNerGY | VOL 6 | JANUARY 2021 | ...,"{""perovskite_composition"": ""(BA)2PbI 4"", ""elec...","{""perovskite_composition"": ""(BA)2PbI 4"", ""elec...",75
3,3,3_52,\t\t\t of 6 RESEARCH | REPORT Downloaded from ...,"{""perovskite_composition"": ""Cs0.05(MA0.10FA0.8...","{""perovskite_composition"": ""Cs0.05(MA0.10FA0.8...",52
4,4,4_26,"Proppe 1,2,10 , Andrew Johnston 2,10 , Sam T...","{""perovskite_composition"": null, ""electron_tra...","{""perovskite_composition"": ""(MAPbBr3)0.05(FAPb...",26


In [4]:
# List to store extracted data
expanded_data = []

for index, row in papers_df.iterrows():
    try:
        row_dict = json.loads(row['output'])  # Convert JSON string to dictionary
    except (json.JSONDecodeError, TypeError):
        continue  # Skip rows where conversion fails

    # Extract common fields
    common_fields = {
        "first_num": row['first_num'],
        "perovskite_composition": row_dict.get("perovskite_composition"),
        "electron_transport_layer": row_dict.get("electron_transport_layer"),
        "hole_transport_layer": row_dict.get("hole_transport_layer"),
        "structure_pin_nip": row_dict.get("structure_pin_nip"),
    }

    # Extract test data
    for key, test_data in row_dict.items():
        if key.startswith("test_") and isinstance(test_data, dict):
            test_row = common_fields.copy()
            test_row["test"] = key  # Store test name
            test_row.update(test_data)  # Merge test details
            expanded_data.append(test_row)

# Convert extracted data to DataFrame
df_expanded = pd.DataFrame(expanded_data)
df_expanded.head()

,first_num,perovskite_composition,electron_transport_layer,hole_transport_layer,structure_pin_nip,test,stability_type,passivating_molecule,humidity,temperature,time,control_pce,treated_pce,control_voc,treated_voc,efficiency_control,efficiency_tret,efficiency_cont
0,0,Cs0.05FA0.85MA0.1PbI3,C60,2PACz and Me-4PACz,PIN,test_1,ISOSL,4-chlorobenzenesulfonate (4Cl-BZS),None,65,1200,24,26.9,None,1.18,None,95,NaN
1,1,None,TinOxide,PTAA,PIN,test_1,ISOST,phenethylammonium,None,85,500,None,19.1,None,1.16,None,None,NaN
2,2,(BA)2PbI 4,tin dioxide,Spiro-OMeTAD,NIP,test_1,ISOSL,None,85,25,1620,22.39,24.35,None,1.185,None,98,58.6
3,2,(BA)2PbI 4,tin dioxide,Spiro-OMeTAD,NIP,test_1_2,ISOSD,None,85,85,1056,None,21.34,None,None,None,94,NaN
4,2,(BA)2PbI 4,tin dioxide,Spiro-OMeTAD,NIP,test_2,ISOSLT,None,None,25,1620,None,24.06,None,None,None,98,NaN


In [7]:
# Selecting columns of interest
columns_of_interest = ['passivating_molecule', 'treated_pce', 'perovskite_composition']

# Will use this DataFrame called 'data' to train the model
data = df_expanded.dropna(subset=columns_of_interest)[columns_of_interest]

---

### SMILES representation and features

In [12]:
# function that converts names into SMILES representation
def fetch_smiles_from_name(molecule_name):
    try:
        # Search for the molecule in PubChem by name
        compounds = pcp.get_compounds(molecule_name, 'name')
        if compounds:
            return compounds[0].isomeric_smiles  # Return the first match's SMILES
        else:
            return np.nan
    except Exception as e:
        print(f"Error fetching SMILES for {molecule_name}: {e}")
        return None

In [14]:
# Cleans string formatting
#### Could be improved upon by looking into string more ####

def fix_unmatched_brackets(s):
    """
    Fixes unmatched brackets in the given string by adding the correct brackets where necessary.

    :param s: Input string with potential unmatched brackets.
    :return: A corrected string with properly balanced brackets.
    """
    opening = "({["
    closing = ")}]"
    match = {')': '(', '}': '{', ']': '['}
    stack = []

    # Step 1: Identify missing closing brackets
    fixed_s = []
    for char in s:
        if char in opening:
            stack.append(char)
            fixed_s.append(char)
        elif char in closing:
            if stack and stack[-1] == match[char]:
                stack.pop()
                fixed_s.append(char)
            else:
                # Add missing opening bracket before unmatched closing
                fixed_s.insert(0, match[char])
                fixed_s.append(char)
        else:
            fixed_s.append(char)

    # Step 2: Add missing closing brackets at the end
    while stack:
        open_bracket = stack.pop()
        fixed_s.append(closing[opening.index(open_bracket)])

    return "".join(fixed_s)


def get_chemical_names(chemical_list):
    cleaned_list = []
    for name in chemical_list:
        # Remove text inside parentheses only if it's extra information (abbreviations)
        name = re.sub(r"\s*\([^)]*\)$", "", name).strip() 
        # Remove spaces after a closing bracket (ensure proper chemical formatting)
        name = re.sub(r"\] +", "]", name)

        cleaned_list.append(name)

    return cleaned_list

In [102]:
def get_smiles(molecule_name):
    base_url = "https://opsin.ch.cam.ac.uk/opsin/"
    smiles_url = base_url + molecule_name + ".smi"
    r = requests.get(smiles_url)
    return r.text if r.status_code == 200 else None

In [116]:
get_smiles("phenylethylammonium lead iodide")


'ClC1=CC=C(C=C1)S(=O)(=O)[O-]'

---
### Composition of the Perovskite

In [22]:
def parse_perovskite_formula(formula):
    # Define allowed species (order matters for multi-letter elements)
    allowed_species = ["FA", "MA", "CS", "Rb", "Pb", "Sn", "I", "Br", "Cl"]

    # if is the nan we return component dictionary with all zeros
    if formula is np.nan:
        formula = ""    
    
    # Dictionary to store parsed results (initialize with 0.0 for all species)
    parsed_result = {species: 0.0 for species in allowed_species}

    # Step 1: Handle groups in parentheses with coefficients (e.g., (FAPbI3)0.95)
    pattern_group = r"\(([^)]+)\)\s*([0-9\.]+)"

    
    
    groups = re.findall(pattern_group, formula)

    if groups:
        for group, coef in groups:
            coef = float(coef)  # Convert coefficient to float
            elements = re.findall(r"(FA|MA|CS|Rb|Pb|Sn|I|Br|Cl)\s*([\d\.]*)", group)
            for element, count in elements:
                count = float(count) if count else 1.0
                parsed_result[element] += count * coef  # Distribute coefficient

    # Step 2: Handle formulas without parentheses (e.g., FA1-xMAxPbI3)
    remaining_formula = re.sub(r"\([^)]*\)\s*[0-9\.]+", "", formula)  # Remove processed groups
    elements = re.findall(r"(FA|MA|CS|Rb|Pb|Sn|I|Br|Cl)\s*([\d\.]*)", remaining_formula)

    for element, count in elements:
        count = float(count) if count and 'x' not in count else 1.0  # Ignore '-x' or 'x'
        parsed_result[element] += count

    # Round to 2 decimal places for all values
    parsed_result = {k: round(v, 2) for k, v in parsed_result.items()}

    return parsed_result

# Test cases
formulas = [
    "(FAPbI3)0.95(MAPbBr3)0.05",
    "FA1-xMAxPbI3",
    "FA0.9CS0.1Rb0.05PbI2.9Br0.1",
    "(CS0.8Rb0.2FAPbI3)0.9(MAPbBr3)0.1",
    "(C4H9NH3)2PbI 4"  # Test case with space
]

for formula in formulas:
    print(f"Formula: {formula}")
    print("Parsed:", parse_perovskite_formula(formula))
    print()

Formula: (FAPbI3)0.95(MAPbBr3)0.05
Parsed: {'FA': 0.95, 'MA': 0.05, 'CS': 0.0, 'Rb': 0.0, 'Pb': 1.0, 'Sn': 0.0, 'I': 2.85, 'Br': 0.15, 'Cl': 0.0}

Formula: FA1-xMAxPbI3
Parsed: {'FA': 1.0, 'MA': 1.0, 'CS': 0.0, 'Rb': 0.0, 'Pb': 1.0, 'Sn': 0.0, 'I': 3.0, 'Br': 0.0, 'Cl': 0.0}

Formula: FA0.9CS0.1Rb0.05PbI2.9Br0.1
Parsed: {'FA': 0.9, 'MA': 0.0, 'CS': 0.1, 'Rb': 0.05, 'Pb': 1.0, 'Sn': 0.0, 'I': 2.9, 'Br': 0.1, 'Cl': 0.0}

Formula: (CS0.8Rb0.2FAPbI3)0.9(MAPbBr3)0.1
Parsed: {'FA': 0.9, 'MA': 0.1, 'CS': 0.72, 'Rb': 0.18, 'Pb': 1.0, 'Sn': 0.0, 'I': 2.7, 'Br': 0.3, 'Cl': 0.0}

Formula: (C4H9NH3)2PbI 4
Parsed: {'FA': 0.0, 'MA': 0.0, 'CS': 0.0, 'Rb': 0.0, 'Pb': 1.0, 'Sn': 0.0, 'I': 4.0, 'Br': 0.0, 'Cl': 0.0}



In [23]:
temp = data['perovskite_composition'].apply(parse_perovskite_formula).apply(pd.Series)
data = data.join(temp)
data

,passivating_molecule,treated_pce,perovskite_composition,passivating_molecule_cleaned,passivating_molecule_SMILES,MolWt,ExactMolWt,LogP,TPSA,NumValenceElectrons,...,Kappa2,FA,MA,CS,Rb,Pb,Sn,I,Br,Cl
0,4-chlorobenzenesulfonate (4Cl-BZS),26.9,Cs0.05FA0.85MA0.1PbI3,4-chlorobenzenesulfonate,C1=CC(=CC=C1S(=O)(=O)[O-])Cl,191.615,190.957516,1.2441,57.20,60.0,...,2.808706,0.85,0.10,0.0,0.00,1.00,0.0,3.00,0.00,0.0
7,phenylethylammonium iodide,18.89,MAPbI 3,phenylethylammonium iodide,C1=CC=C(C=C1)CC[NH3+].[I-],249.095,249.001447,-2.5250,27.64,56.0,...,5.676536,0.00,1.00,0.0,0.00,1.00,0.0,3.00,0.00,0.0
14,butylammonium bromide,19.8,Cs0.15FA0.85PbI2.19Br0.81,butylammonium bromide,CCCCN.Br,154.051,153.015311,1.3231,26.02,40.0,...,9.062499,0.85,0.00,0.0,0.00,1.00,0.0,2.19,0.81,0.0
15,2-thiopheneethylammonium chloride,23.8,Cs0.12FA0.8MA0.08PbI1.8Br1,2-thiopheneethylammonium chloride,C1=CSC(=C1)CCN.Cl,163.673,163.022248,1.6711,26.02,52.0,...,4.793919,0.80,0.08,0.0,0.00,1.00,0.0,1.80,1.00,0.0
31,phenethylammonium iodide,23.91,Cs0.05MA0.1FA0.85PbI3,phenethylammonium iodide,C1=CC=C(C=C1)CCN.I,249.095,249.001447,1.8058,26.02,56.0,...,5.676536,0.85,0.10,0.0,0.00,1.00,0.0,3.00,0.00,0.0
36,3-(aminomethyl)pyridine,25.49,Rb0.05Cs0.05MA0.05FA0.85Pb(I0.95Br0.05)3,3-(aminomethyl)pyridine,C1=CC(=CN=C1)CN,108.144,108.068748,0.5403,38.91,42.0,...,2.425724,0.85,0.05,0.0,0.05,1.00,0.0,2.85,0.15,0.0
42,2-thiophenemethylammonium bromide,20.82,(FAPbI 3 ) 0.87(MAPbBr3)0.13]0.92(CsPbI3)0.08,2-thiophenemethylammonium bromide,C1=CSC(=C1)CN.Br,194.097,192.956082,1.7847,26.02,46.0,...,4.062432,0.87,0.13,0.0,0.00,1.08,0.0,2.85,0.39,0.0
50,Lithium Fluoride,19.6,CsPbI0.05[(FAPbI3)0.89(MAPbBr3)0.11]0.95,Lithium Fluoride,[Li+].[F-],25.939,26.014408,-5.9920,0.00,8.0,...,1.527403,0.89,0.11,0.0,0.00,2.00,0.0,2.72,0.33,0.0
56,butylammonium,21,(MA0.14FA0.81Cs0.05)9-Pb10(I0.85Br0.15),butylammonium,CCCC[NH3+],74.147,74.096426,0.0284,27.64,32.0,...,3.960000,7.29,1.26,0.0,0.00,10.00,0.0,0.85,0.15,0.0
66,phenethylammonium iodide,19.07,MAPbI3,phenethylammonium iodide,C1=CC=C(C=C1)CCN.I,249.095,249.001447,1.8058,26.02,56.0,...,5.676536,0.00,1.00,0.0,0.00,1.00,0.0,3.00,0.00,0.0


In [24]:
data.to_csv('data.csv')

---
## Model Building

In [36]:
data.columns

Index(['passivating_molecule', 'treated_pce', 'perovskite_composition',
       'passivating_molecule_cleaned', 'passivating_molecule_SMILES', 'MolWt',
       'ExactMolWt', 'LogP', 'TPSA', 'NumValenceElectrons', 'NumRotBonds',
       'NumHBA', 'NumHBD', 'FractionCSP3', 'AromaticRings', 'SaturatedRings',
       'Heteroatoms', 'HeavyAtoms', 'SpiroAtoms', 'BridgeheadAtoms',
       'FpDensityMorgan1', 'FpDensityMorgan2', 'FpDensityMorgan3', 'QED',
       'LipinskiHBA', 'LipinskiHBD', 'NumRings', 'NumAmideBonds', 'BalabanJ',
       'BertzCT', 'Chi0', 'Chi1', 'Chi2n', 'Kappa1', 'Kappa2', 'FA', 'MA',
       'CS', 'Rb', 'Pb', 'Sn', 'I', 'Br', 'Cl'],
      dtype='object')

In [48]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score

# Define feature matrix X and target variable y
X = data.drop(columns=["perovskite_composition", "passivating_molecule", 
                       "passivating_molecule_cleaned", "passivating_molecule_SMILES", "treated_pce"])
y = data["treated_pce"]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define models and hyperparameter grids
models = {
    "Ridge": (Ridge(), {"alpha": [0.1, 1, 10, 100]}),
    "Lasso": (Lasso(), {"alpha": [0.1, 1, 10, 100]}),
    "SVR": (SVR(), {"C": [0.1, 1, 10], "gamma": ["scale", "auto"], "kernel": ["rbf", "linear"]}),
    "RandomForest": (RandomForestRegressor(), {"n_estimators": [50, 100, 200], "max_depth": [None, 10, 20]}),
    "GradientBoosting": (GradientBoostingRegressor(), {"n_estimators": [50, 100, 200], "learning_rate": [0.01, 0.1, 0.2]})
}

# Store results
results = []

# Train and evaluate models
for name, (model, param_grid) in models.items():
    pipeline = Pipeline([("scaler", StandardScaler()), ("model", model)])
    grid_search = GridSearchCV(pipeline, {"model__" + key: value for key, value in param_grid.items()}, cv=5, scoring="neg_mean_squared_error")
    grid_search.fit(X_train, y_train)
    
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)
    
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({"Model": name, "Best Params": grid_search.best_params_, "MSE": mse, "R2": r2})

# Convert results to a DataFrame and display
results_df = pd.DataFrame(results)
results_df

,Model,Best Params,MSE,R2
0,Ridge,{'model__alpha': 100},52.172208,-1.416694
1,Lasso,{'model__alpha': 1},51.424064,-1.382039
2,SVR,"{'model__C': 10, 'model__gamma': 'auto', 'mode...",22.281307,-0.032103
3,RandomForest,"{'model__max_depth': 10, 'model__n_estimators'...",16.017289,0.258055
4,GradientBoosting,"{'model__learning_rate': 0.2, 'model__n_estima...",10.779203,0.500691


In [76]:
data.drop(columns=["perovskite_composition", "passivating_molecule", 
                       "passivating_molecule_cleaned", "passivating_molecule_SMILES", "treated_pce"])

,MolWt,ExactMolWt,LogP,TPSA,NumValenceElectrons,NumRotBonds,NumHBA,NumHBD,FractionCSP3,AromaticRings,...,Kappa2,FA,MA,CS,Rb,Pb,Sn,I,Br,Cl
0,191.615,190.957516,1.2441,57.20,60.0,1.0,3.0,0.0,0.000000,1.0,...,2.808706,0.85,0.10,0.0,0.00,1.00,0.0,3.00,0.00,0.0
7,249.095,249.001447,-2.5250,27.64,56.0,2.0,0.0,1.0,0.250000,1.0,...,5.676536,0.00,1.00,0.0,0.00,1.00,0.0,3.00,0.00,0.0
14,154.051,153.015311,1.3231,26.02,40.0,2.0,1.0,1.0,1.000000,0.0,...,9.062499,0.85,0.00,0.0,0.00,1.00,0.0,2.19,0.81,0.0
15,163.673,163.022248,1.6711,26.02,52.0,2.0,2.0,1.0,0.333333,1.0,...,4.793919,0.80,0.08,0.0,0.00,1.00,0.0,1.80,1.00,0.0
31,249.095,249.001447,1.8058,26.02,56.0,2.0,1.0,1.0,0.250000,1.0,...,5.676536,0.85,0.10,0.0,0.00,1.00,0.0,3.00,0.00,0.0
36,108.144,108.068748,0.5403,38.91,42.0,1.0,2.0,1.0,0.166667,1.0,...,2.425724,0.85,0.05,0.0,0.05,1.00,0.0,2.85,0.15,0.0
42,194.097,192.956082,1.7847,26.02,46.0,1.0,2.0,1.0,0.200000,1.0,...,4.062432,0.87,0.13,0.0,0.00,1.08,0.0,2.85,0.39,0.0
50,25.939,26.014408,-5.9920,0.00,8.0,0.0,0.0,0.0,0.000000,0.0,...,1.527403,0.89,0.11,0.0,0.00,2.00,0.0,2.72,0.33,0.0
56,74.147,74.096426,0.0284,27.64,32.0,2.0,0.0,1.0,1.000000,0.0,...,3.960000,7.29,1.26,0.0,0.00,10.00,0.0,0.85,0.15,0.0
66,249.095,249.001447,1.8058,26.02,56.0,2.0,1.0,1.0,0.250000,1.0,...,5.676536,0.00,1.00,0.0,0.00,1.00,0.0,3.00,0.00,0.0


In [80]:
non_numeric_columns = data.select_dtypes(exclude=[np.number]).columns.tolist()
non_numeric_columns 

['passivating_molecule',
 'treated_pce',
 'perovskite_composition',
 'passivating_molecule_cleaned',
 'passivating_molecule_SMILES']

In [90]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import Ridge, Lasso
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression


# Identify non-numeric columns
data['treated_pce'] = data['treated_pce'].astype(float)

# Define feature matrix X and target variable y
X = data.drop(columns=["perovskite_composition", "passivating_molecule", 
                       "passivating_molecule_cleaned", "passivating_molecule_SMILES", "treated_pce"] + non_numeric_columns)
y = pd.to_numeric(data["treated_pce"], errors="coerce")

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Handle NaN values in target variable
y = y.dropna()
X_scaled = X_scaled[:len(y)]  # Ensure feature matrix matches target variable size

# Perform feature selection (keeping top 10 features based on f-score)
selector = SelectKBest(score_func=f_regression, k=min(10, X.shape[1]))
X_selected = selector.fit_transform(X_scaled, y)

# Define models
models = {
    "Ridge": Ridge(alpha=1.0),
    "Lasso": Lasso(alpha=0.1),
    "SVR": SVR(C=1.0, kernel="linear")
}

# Use K-Fold with k=5
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Store results
results = []

for name, model in models.items():
    pipeline = Pipeline([("scaler", StandardScaler()), ("model", model)])
    scores = cross_val_score(pipeline, X_selected, y, cv=kf, scoring="r2")
    
    mean_r2 = np.mean(scores)
    std_r2 = np.std(scores)
    
    results.append({"Model": name, "Mean R²": mean_r2, "Std R²": std_r2})

# Convert results to a DataFrame and display
results_df = pd.DataFrame(results)
print(results_df)

   Model    Mean R²     Std R²
0  Ridge -12.335463  24.374152
1  Lasso  -6.049710  11.843006
2    SVR  -5.781895  11.192918
